In [1]:
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

This code downloads one's spotify library for clustering - Code adapted from https://towardsdatascience.com/k-means-clustering-using-spotify-song-features-9eb7d53d105c

In [4]:
scope = "user-library-read playlist-modify-private"
OAuth = SpotifyOAuth(
        scope=scope,         
        redirect_uri='http://localhost:5672/callback',
        client_id = 'd7fed0d1ecac4b5184554706d76d5a22',
        client_secret = '0c1758d9564b4659b730f2a05c104ccb',
        username= 'nikim99')

sp = spotipy.Spotify(auth_manager=OAuth)

In [5]:
offset = 0
songs = []
items = []
ids = []
while True:
    content = sp.current_user_saved_tracks(limit=50, offset=offset)
    songs += content['items']
    if content['next'] is not None:
        offset += 100
    else:
        break

In [6]:
# sample song
songs[0]
len(songs)

595

In [7]:
song_names = []
artist_names = []
audio_features = []
index = 0
for i in songs:
    song_names.append(i['track']['name'])
    artist_names.append(i['track']['artists'][0]['name'])
    ids.append(i['track']['id'])
while index < len(ids):
    audio_features += sp.audio_features(ids[index:index + 50])
    index += 50
    
features_list = []
for features in audio_features:
    features_list.append([features['danceability'], features['energy'],
                          features['key'],features['loudness'],
                          features['mode'], features['speechiness'],
                          features['acousticness'],features['instrumentalness'],
                          features['liveness'], features['valence'],
                          features['tempo'],features['duration_ms'],
                          features['time_signature'], features['type'],
                          features['uri']])

df = pd.DataFrame(features_list, columns= ["danceability", "energy", "key", "loudness", "mode",
             "speechiness", "acousticness", "instrumentalness",
             "liveness", "valence", "tempo", "duration_ms", "time_signature", "type", "uri"])


In [8]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,type,uri
0,0.689,0.785,1,-5.792,0,0.1750,0.183000,0.000744,0.0579,0.532,145.860,162707,4,audio_features,spotify:track:3DfTZtNNSDBV10YMu9JFWF
1,0.510,0.536,1,-5.988,1,0.0399,0.021700,0.000007,0.0869,0.384,90.763,172880,4,audio_features,spotify:track:4GSqiK9YdVNhilXMFtCiQB
2,0.322,0.557,0,-6.055,1,0.0473,0.519000,0.000002,0.1130,0.212,99.898,202950,4,audio_features,spotify:track:3MNlGZIqeH2HEcMUItoxes
3,0.659,0.467,5,-8.526,1,0.0810,0.143000,0.000000,0.0776,0.919,170.029,135500,4,audio_features,spotify:track:7eFIg4OmtXa0Wf5eeFOLs4
4,0.732,0.551,10,-6.181,0,0.0242,0.481000,0.000000,0.1070,0.262,93.997,155537,4,audio_features,spotify:track:5LoHGfaG1rnEnGY3PCH2FA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,0.511,0.950,4,-3.833,0,0.1840,0.061200,0.000000,0.2030,0.539,151.943,212467,4,audio_features,spotify:track:49nmsafpkJp2lDj9b4jkxh
591,0.556,0.851,6,-2.981,0,0.0443,0.000045,0.000020,0.2110,0.271,114.948,211027,4,audio_features,spotify:track:05Z77MHLPcBA2Wd5eDSJzl
592,0.571,0.926,8,-3.576,0,0.0479,0.000217,0.003760,0.0522,0.576,133.903,226720,4,audio_features,spotify:track:3iJmiaoZWzykf3Hl4IeYtI
593,0.474,0.853,8,-3.775,1,0.1200,0.000172,0.001210,0.1090,0.559,170.076,241600,4,audio_features,spotify:track:6VOWUjl7sumb6IpBPkjQat


In [9]:
print(len(song_names), len(artist_names))
print(song_names[0:10])
print(artist_names[0:10])

595 595
['Last Laugh - From "Promising Young Woman" Soundtrack', 'Devil I Know', 'Funeral', 'Deal With The Devil', 'Crystal Ball', 'Rain (from The Suicide Squad)', 'Don’t Blame Me', 'killer queen', 'Desperado', 'Boyfriend']
['FLETCHER', 'Allie X', 'Rachel Lorin', 'Rory Webley', 'Sebastian Javier', 'grandson', 'Taylor Swift', 'Mad Tsai', 'Rihanna', 'Dove Cameron']


In [10]:
df['name'] = song_names
df['artists'] = [["'" + i + "'"] for i in artist_names]

In [12]:
#df
df.to_csv('tracks_features_niki.csv')